# Transfer Learning of PreTrained VGG16 on ImageNet

## Load augmented images dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!cd /content/drive/MyDrive/Colab\ Notebooks/Cars/

In [ ]:
!git clone https://github.com/ibalzuhairi/Cars_Classifiers.git

fatal: destination path 'Cars_Classifiers' already exists and is not an empty directory.


In [ ]:
#!wget http://www.awlaa.app/cars-dataset.zip 
!wget http://www.awlaa.app/dataset-aug3.zip 

--2021-11-25 00:16:15--  http://www.awlaa.app/dataset-aug3.zip
Resolving www.awlaa.app (www.awlaa.app)... 51.144.85.121
Connecting to www.awlaa.app (www.awlaa.app)|51.144.85.121|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.awlaa.app/dataset-aug3.zip [following]
--2021-11-25 00:16:15--  https://www.awlaa.app/dataset-aug3.zip
Connecting to www.awlaa.app (www.awlaa.app)|51.144.85.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4712338562 (4.4G) [application/zip]
Saving to: ‘dataset-aug3.zip’

dataset-aug3.zip    100%[===================>]   4.39G  21.0MB/s    in 3m 39s  

2021-11-25 00:19:54 (20.6 MB/s) - ‘dataset-aug3.zip’ saved [4712338562/4712338562]



In [ ]:
!unzip dataset-aug3.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset-aug/train_spl/95/aug-007777.jpg_0_7725.jpg  
  inflating: dataset-aug/train_spl/95/aug-007777.jpg_0_9685.jpg  
  inflating: dataset-aug/train_spl/95/aug-007778.jpg_0_1006.jpg  
  inflating: dataset-aug/train_spl/95/aug-007778.jpg_0_2157.jpg  
  inflating: dataset-aug/train_spl/95/aug-007778.jpg_0_2224.jpg  
  inflating: dataset-aug/train_spl/95/aug-007778.jpg_0_5020.jpg  
  inflating: dataset-aug/train_spl/95/aug-007778.jpg_0_824.jpg  
  inflating: dataset-aug/train_spl/95/aug-007779.jpg_0_2492.jpg  
  inflating: dataset-aug/train_spl/95/aug-007779.jpg_0_2927.jpg  
  inflating: dataset-aug/train_spl/95/aug-007779.jpg_0_3092.jpg  
  inflating: dataset-aug/train_spl/95/aug-007779.jpg_0_4899.jpg  
  inflating: dataset-aug/train_spl/95/aug-007779.jpg_0_8082.jpg  
  inflating: dataset-aug/train_spl/95/aug-007782.jpg_0_3095.jpg  
  inflating: dataset-aug/train_spl/95/aug-007782.jpg_0_7069.jpg  
  inflating: dataset-aug/t

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
keras.__version__

'2.7.0'

In [ ]:
train_path = './dataset-aug/train_spl'
valid_path = './dataset-aug/val_spl'
test_path = './dataset-aug/test_spl'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=500)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=500)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=500, shuffle=False)


FileNotFoundError: ignored

In [2]:
new_input = tf.keras.Input(shape=(224, 224, 3))
model_original = tf.keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
# freeze trained weight
for layer in model_original.layers:
      layer.trainable = False

NameError: name 'tf' is not defined

In [1]:
# Add NN on top of the model
flat1 = tf.keras.layers.Flatten()(model_original.layers[-1].output)
class1 = Dense(512, activation='relu' ,kernel_regularizer=tf.keras.regularizers.l2(0.01))(flat1)
class2 = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(class1)
output = Dense(196, activation='softmax')(class2)

NameError: name 'tf' is not defined

In [ ]:
model = Model(inputs=model_original.input, outputs=output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Colab\ Notebooks/Cars/M_VGG1_checkpoint-{epoch:02d}.hdf5", monitor='loss', verbose=0,
  save_best_only=True, mode='auto', save_freq='epoch')
#checkpoint = ModelCheckpoint("M_VGG1_checkpoint-{epoch:02d}.hdf5", monitor='loss', verbose=0,
 # save_best_only=True, mode='auto', save_freq='epoch')

In [ ]:
csv_logger = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/Colab\ Notebooks/Cars/trainingImageNetRegL20.01VGGVolab-smallnet.csv')
#csv_logger = tf.keras.callbacks.CSVLogger('trainingImageNetRegL20.01VGGVolab.csv')

In [ ]:
H = model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=400,
            verbose=1,callbacks=[checkpoint,csv_logger]
)

Epoch 1/400
